In [2]:
from keras.preprocessing import sequence
import pandas as pd
import numpy as np

/home/huypham/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_csv('train_encode.csv')

In [4]:
df.head()

,descVec,wordIdVec
0,"[2427, 3953, 4687, 2416, 2337, 772, 40, 4878, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[2427, 3209, 3954, 4687, 0, 3730, 4945, 40, 12...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[2427, 3209, 3954, 4687, 2195, 2433, 3324, 259...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[2427, 5288, 4809, 4093, 5335, 3253, 872, 3324...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[2799, 2031, 380, 3412, 772, 183, 40, 2061, 24...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
max([len(i) for i in df.descVec])

270

In [17]:
maxlen = 300
max_features = 6000

In [18]:
input_train = [list(map(int, i.strip('[').strip(']').split(','))) for i in df.descVec]
y_train = [list(map(int, i[1:-1].replace(', ', ' ').split(' '))) for i in df.wordIdVec]
input_train_pad = sequence.pad_sequences(input_train, maxlen=maxlen, padding='post')
y_train_pad = sequence.pad_sequences(y_train, maxlen=maxlen, padding='post')

In [19]:
input_train_pad.shape

(9649, 300)

In [20]:
testdf = pd.read_csv('test_encode.csv')

input_test = [list(map(int, i.strip('[').strip(']').split(','))) for i in testdf.descVec]
y_test = [list(map(float, i[1:-1].replace(', ', ' ').split(' '))) for i in testdf.wordIdVec]
input_test_pad = sequence.pad_sequences(input_test, maxlen=maxlen, padding='post')
y_test_pad = sequence.pad_sequences(y_test, maxlen=maxlen, padding='post')

In [21]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Embedding, Dropout, Activation, TimeDistributed

In [24]:
model = Sequential()
model.add(Embedding(6000, 32))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(maxlen, activation='softmax'))
#model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['categorical_accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 32)          192000    
_________________________________________________________________
lstm_9 (LSTM)                (None, None, 64)          24832     
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 300)               19500     
Total params: 269,356
Trainable params: 269,356
Non-trainable params: 0
_________________________________________________________________


In [25]:
#inp = input_train_pad.reshape(1, 9649,maxlen)
res = model.fit(input_train_pad, y_train_pad, epochs = 10, batch_size=128, validation_split=0.2)

Train on 7719 samples, validate on 1930 samples
Epoch 1/10
7719/7719 [==============================] - 48s 6ms/step - loss: 3.7304 - categorical_accuracy: 0.2384 - val_loss: 2.5132 - val_categorical_accuracy: 0.3187
Epoch 2/10
7719/7719 [==============================] - 48s 6ms/step - loss: 2.6617 - categorical_accuracy: 0.2255 - val_loss: 2.4824 - val_categorical_accuracy: 0.3187
Epoch 3/10
7719/7719 [==============================] - 53s 7ms/step - loss: 2.6003 - categorical_accuracy: 0.2442 - val_loss: 2.4787 - val_categorical_accuracy: 0.3187
Epoch 4/10
7719/7719 [==============================] - 56s 7ms/step - loss: 2.5845 - categorical_accuracy: 0.2570 - val_loss: 2.4719 - val_categorical_accuracy: 0.3187
Epoch 5/10
7719/7719 [==============================] - 59s 8ms/step - loss: 2.5655 - categorical_accuracy: 0.2629 - val_loss: 2.4714 - val_categorical_accuracy: 0.3187
Epoch 6/10
7719/7719 [==============================] - 54s 7ms/step - loss: 2.5515 - categorical_accuracy:

## lstm 32: 
Epoch 1/1
7719/7719 [==============================] - 26s 3ms/step - loss: 4.9358 - categorical_accuracy: 0.1411 - val_loss: 2.9808 - val_categorical_accuracy: 0.3187

## LSTM 64:
7719/7719 [==============================] - 37s 5ms/step - loss: 4.1176 - categorical_accuracy: 0.2493 - val_loss: 2.5061 - val_categorical_accuracy: 0.3187

## LSTM 128:
7719/7719 [==============================] - 82s 11ms/step - loss: 3.5538 - categorical_accuracy: 0.2209 - val_loss: 2.4833 - val_categorical_accuracy: 0.3187

## LSTM 32 x 4 layers:
7719/7719 [==============================] - 98s 13ms/step - loss: 4.7243 - categorical_accuracy: 0.1021 - val_loss: 2.9010 - val_categorical_accuracy: 0.0731

## LSTM 64 x 3:
7719/7719 [==============================] - 121s 16ms/step - loss: 3.8503 - categorical_accuracy: 0.1954 - val_loss: 2.5308 - val_categorical_accuracy: 0.3187

## LSTM 64 x 2 layers -> best:
7719/7719 [==============================] - 81s 10ms/step - loss: 4.0335 - categorical_accuracy: 0.2546 - val_loss: 2.5199 - val_categorical_accuracy: 0.3187

## LSTM 64 x 2 layers - 10 epochs:
Epoch 10/10
7719/7719 [==============================] - 82s 11ms/step - loss: 2.5154 - categorical_accuracy: 0.3227 - val_loss: 2.4672 - val_categorical_accuracy: 0.3187



In [30]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(maxlen, activation='relu'))
#model.add(Activation('softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics =['acc'])
model.summary()
#inp = input_train_pad.reshape(1, 9649,maxlen)
res = model.fit(input_train_pad, y_train_pad, epochs = 10, batch_size=128, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 32)          192000    
_________________________________________________________________
lstm_13 (LSTM)               (None, None, 64)          24832     
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 300)               19500     
Total params: 269,356
Trainable params: 269,356
Non-trainable params: 0
_________________________________________________________________
Train on 7719 samples, validate on 1930 samples
Epoch 1/10
7719/7719 [==============================] - 47s 6ms/step - loss: 3.6325 - ac

In [162]:
input_test_pad.shape

(2413, 500)

In [163]:
y_test_pred = model.predict(input_test_pad)

In [165]:
y_test_pred.shape

(2413, 500)

In [167]:
y_test_pad.shape

(2413, 500)

In [177]:
y_test_pred_nopad = [y_test_pred[:len(i)] for i in input_test]

In [147]:
#list(y_test_pred_nopad[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [178]:
#from sklearn.metrics import accuracy_score
acc=0
for i, j in zip(list(y_test), list(y_test_pred_nopad)):
    if list(i)==list(j):
        acc += 1
        
print(acc/len(y_test_pad))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
su = 0
for i in y_test_pad:
    if sum(i) != 0:
        su += 1
print(su)

In [ ]:
# 0.14022178464089544

In [180]:
y_test_pred_nopad.shape

AttributeError: 'list' object has no attribute 'shape'

In [173]:
[0,0] == [0,0]

True

# Y TRAIN pred

In [26]:
y_train_pred = model.predict(input_train_pad)

In [27]:
y_train_pred.shape

(9649, 300)

In [28]:
y_train_pred[0]

array([1.84258357e-01, 5.00209518e-02, 7.55441841e-03, 5.62371612e-02,
       1.55574635e-01, 1.20110124e-01, 5.99166192e-02, 8.54462460e-02,
       6.19351491e-02, 6.58268258e-02, 3.79804261e-02, 3.06578595e-02,
       2.62765158e-02, 1.11069800e-02, 9.68732219e-03, 5.89362765e-03,
       6.72245491e-03, 9.15608928e-03, 4.57697315e-03, 1.09240168e-03,
       1.89800817e-03, 3.41036310e-03, 2.15204433e-03, 5.23608469e-04,
       6.92099493e-05, 6.82335158e-05, 2.73633952e-04, 1.77113259e-06,
       6.59047873e-06, 5.69996246e-06, 6.95252675e-06, 1.94289503e-04,
       2.51528309e-06, 5.34686296e-06, 4.78694028e-06, 2.15346176e-06,
       2.50296216e-06, 4.46290642e-06, 3.21452899e-06, 5.67579718e-06,
       4.99858015e-06, 4.37822200e-06, 4.09709992e-06, 4.38858933e-06,
       8.28472912e-05, 4.73047112e-06, 4.70773011e-06, 6.03309672e-06,
       6.24362065e-06, 5.83162819e-06, 3.82964345e-06, 2.69262569e-06,
       6.32536921e-06, 6.30908471e-06, 2.77333925e-06, 6.57808368e-06,
      

# Y TEST pred

In [ ]:
import pandas as pd
import csv
from utils import *

vocab = pd.read_csv('vocab.csv', na_filter = False)
vocab = list(vocab.vocab)

df = pd.read_csv('test.csv', index_col=0)
df.desc = [textPrep(i) for i in df.desc]
df.wordId = [textPrep(i) for i in df.wordId]

wordIdVec_rev = [getWordIdenFromOutputVector(originSt=list(df.desc)[i], outVec=y_test_pad[i]) for i in range(len(wordIdVec))]